# Tokenizers

Tokenizer'lar NLP pipeline'nın temel bileşenlerinden biridir. Tek bir amaca hizmet ederler: **metni model tarafından işlenebilecek verilere çevirmek**. Modeller yalnızca sayıları işleyebilir, bu nedenle tokenizer'ların metin girdilerimizi sayısal verilere dönüştürmesi gerekir. Bu bölümde, tokenizasyon işlem hattında tam olarak ne olduğunu inceleyeceğiz.

NLP görevlerinde, genellikle işlenen veriler ham metindir. İşte böyle bir metin örneği:

> Jim Henson was a puppeteer

Ancak, modeller yalnızca sayıları işleyebilir, bu nedenle ham metni sayılara dönüştürmenin bir yolunu bulmamız gerekir. Tokenizer'ların yaptığı da budur ve bunu yapmanın pek çok yolu vardır. Amaç, en anlamlı temsili - yani model için en anlamlı olanı - ve mümkünse en küçük temsili bulmaktır.

Şimdi bazı tokenizasyon algoritması örneklerine bir göz atalım ve tokenizasyon hakkında aklınıza gelebilecek bazı soruları yanıtlamaya çalışalım.

## Word-based

Akla gelen ilk tokenizer türü kelime tabanlıdır. Genellikle yalnızca birkaç kuralla kurulumu ve kullanımı çok kolaydır ve genellikle iyi sonuçlar verir. Örneğin, aşağıdaki resimde amaç ham metni kelimelere ayırmak ve her biri için sayısal bir temsil bulmaktır:

![image](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/word_based_tokenization.svg)

Metni bölmenin farklı yolları vardır. Örneğin, Python'un **split()** fonksiyonunu uygulayarak metni kelimelere ayırmak için boşlukları kullanabiliriz:

In [1]:
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)

['Jim', 'Henson', 'was', 'a', 'puppeteer']


Noktalama işaretleri için ekstra kurallara sahip kelime tokenizer çeşitleri de vardır. Bu tür bir tokenizer ile oldukça büyük "kelime hazineleri" elde edebiliriz; burada bir kelime hazinesi, külliyatımızda sahip olduğumuz toplam bağımsız token sayısı ile tanımlanır.

Her kelimeye 0'dan başlayıp kelime dağarcığının boyutuna kadar giden bir kimlik atanır. Model, her kelimeyi tanımlamak için bu kimlikleri kullanır.

Kelime tabanlı bir belirteçleyici ile bir dili tamamen kapsamak istiyorsak, dildeki her kelime için bir tanımlayıcıya sahip olmamız gerekir ki bu da çok büyük miktarda token üretecektir. Örneğin, İngilizce dilinde 500.000'den fazla kelime vardır, bu nedenle her kelimeden bir girdi kimliğine bir harita oluşturmak için bu kadar çok kimliği takip etmemiz gerekir. Ayrıca, "dog" gibi kelimeler "dogs" gibi kelimelerden farklı şekilde temsil edilir ve modelin başlangıçta "dog" ve "dogs" kelimelerinin benzer olduğunu bilmesinin hiçbir yolu yoktur: iki kelimeyi ilgisiz olarak tanımlayacaktır. Aynı durum, modelin başlangıçta benzer olarak görmeyeceği "run" ve "running" gibi diğer benzer kelimeler için de geçerlidir.

Son olarak, kelime dağarcığımızda olmayan kelimeleri temsil etmek için özel bir belirtece ihtiyacımız var. Bu, "bilinmeyen" token olarak bilinir ve genellikle "[UNK]" veya "\<unk\>" olarak gösterilir. Tokenizer'ın bu tokenlardan çok sayıda ürettiğini görürseniz bu genellikle kötüye işarettir, çünkü bir kelimenin mantıklı bir temsilini alamamıştır ve yol boyunca bilgi kaybediyorsunuzdur. Sözcük dağarcığını oluştururken amaç, tokenizer'ın mümkün olduğunca az sayıda sözcüğü bilinmeyen tokena dönüştürmesini sağlamaktır.

Bilinmeyen token miktarını azaltmanın bir yolu, karakter tabanlı bir tokenizer kullanarak bir seviye daha derine inmektir.

## Character-based

Karakter tabanlı tokenizer'lar metni kelimeler yerine karakterlere böler. Bunun iki temel faydası vardır:

- Sözcük dağarcığı çok daha küçüktür.
- Her kelime karakterlerden oluşturulabildiği için çok daha az kelime dağarcığı dışı (bilinmeyen) token vardır.

Ancak burada da boşluklar ve noktalama işaretleriyle ilgili bazı sorular ortaya çıkmaktadır:

![eimag](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/character_based_tokenization.svg)

Bu yaklaşım da mükemmel değildir. Temsil artık kelimeler yerine karakterlere dayandığından, sezgisel olarak daha az anlamlı olduğu iddia edilebilir: her bir karakter tek başına çok fazla anlam ifade etmez, oysa kelimeler için durum böyledir. Ancak bu durum yine dile göre farklılık göstermektedir; örneğin Çince'de her bir karakter Latin dilindeki bir karakterden daha fazla bilgi taşımaktadır.

Dikkate alınması gereken bir diğer husus da modelimiz tarafından işlenecek çok büyük miktarda token elde edeceğimizdir: kelime tabanlı bir tokenizer ile bir kelime yalnızca tek bir token olurken, karakterlere dönüştürüldüğünde kolayca 10 veya daha fazla tokena dönüşebilir.

Her iki dünyanın da en iyisini elde etmek için, iki yaklaşımı birleştiren üçüncü bir teknik kullanabiliriz: **subword tokenization**.

## Subword tokenization

Subword tokenization algoritmaları, **sık kullanılan kelimelerin daha küçük alt kelimelere bölünmemesi gerektiği**, **ancak nadir kelimelerin anlamlı alt kelimelere ayrıştırılması gerektiği ilkesine dayanır**.

Örneğin, "annoyingly" nadir bir kelime olarak kabul edilebilir ve "annoying" ve "ly" olarak ayrıştırılabilir. Bunların her ikisinin de bağımsız alt kelimeler olarak daha sık görünmesi muhtemeldir, ancak aynı zamanda "annoyingly" kelimesinin anlamı "annoying" ve "ly" kelimelerinin bileşik anlamı tarafından korunur.

Aşağıda, bir alt kelime tokenizasyon algoritmasının "Let's do tokenization!" dizisini nasıl tokenize edeceğini gösteren bir örnek verilmiştir:

![image](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/bpe_subword.svg)

Bu alt sözcükler çok fazla semantik anlam sağlamaktadır: örneğin, yukarıdaki örnekte "tokenization" sözcüğü "token" ve "ization" sözcüklerine bölünmüştür; bu iki sözcük hem semantik anlam taşımakta hem de alan açısından verimli olmaktadır (uzun bir sözcüğü temsil etmek için yalnızca iki token gereklidir). Bu, küçük kelime dağarcıklarıyla nispeten iyi bir kapsama alanına sahip olmamızı ve neredeyse hiç bilinmeyen token olmamasını sağlar.

Bu yaklaşım özellikle, alt kelimeleri bir araya getirerek (neredeyse) keyfi olarak uzun karmaşık kelimeler oluşturabileceğiniz Türkçe gibi sondan eklemeli dillerde kullanışlıdır.

### Ve daha fazlası!

Şaşırtıcı olmayan bir şekilde, daha birçok teknik var. Birkaçını saymak gerekirse:

- GPT-2'de kullanılan bayt düzeyinde BPE
- WordPiece, BERT'te kullanıldığı gibi
- Birkaç çok dilli modelde kullanıldığı gibi SentencePiece veya Unigram

Artık API'yi kullanmaya başlamak için tokenizer'ların nasıl çalıştığı hakkında yeterli bilgiye sahip olmalısınız.

## Loading and saving

Tokenizer'ları yüklemek ve kaydetmek modellerde olduğu kadar basittir. Aslında, aynı iki metoda dayanır: from_pretrained() ve save_pretrained(). Bu metotlar, tokenizer tarafından kullanılan algoritmayı (modelin mimarisi gibi) ve kelime dağarcığını (modelin ağırlıkları gibi) yükler veya kaydeder.

BERT ile aynı kontrol noktası ile eğitilen BERT tokenizer'ın yüklenmesi, BertTokenizer sınıfını kullanmamız dışında, modelin yüklenmesi ile aynı şekilde yapılır:

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

AutoModel'e benzer şekilde, AutoTokenizer sınıfı, kontrol noktası adına göre kütüphanedeki uygun tokenizer sınıfını alır ve herhangi bir kontrol noktasıyla doğrudan kullanılabilir:

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Artık tokenizer'ı önceki bölümde gösterildiği gibi kullanabiliriz:

In [4]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

Bölüm 3'te token_type_ids hakkında daha fazla konuşacağız ve attention_mask anahtarını biraz sonra açıklayacağız. İlk olarak, input_id'lerin nasıl oluşturulduğunu görelim. Bunu yapmak için, tokenizer'ın ara yöntemlerine bakmamız gerekecek.

## Encoding

Metni sayılara çevirmek kodlama olarak bilinir. Kodlama iki aşamalı bir işlemle yapılır: tokenizasyon ve ardından giriş kimliklerine dönüştürme.

Gördüğümüz gibi, ilk adım metni genellikle token olarak adlandırılan kelimelere (veya kelimelerin bölümlerine, noktalama işaretlerine vb.) ayırmaktır. Bu süreci yönetebilecek birden fazla kural vardır, bu nedenle model ön eğitimden geçirilirken kullanılan kuralların aynısını kullandığımızdan emin olmak için modelin adını kullanarak tokenizer'ı örneklememiz gerekir.

İkinci adım, bu tokenları sayılara dönüştürmektir, böylece onlardan bir tensör oluşturabilir ve modele besleyebiliriz. Bunu yapmak için, tokenizer'ın bir kelime dağarcığı vardır; bu, onu from_pretrained() yöntemiyle örneklediğimizde indirdiğimiz kısımdır. Yine, model ön eğitimden geçirilirken kullanılan kelime dağarcığının aynısını kullanmamız gerekir.

İki adımı daha iyi anlamak için bunları ayrı ayrı inceleyeceğiz. Size bu adımların ara sonuçlarını göstermek için tokenizasyon pipeline'ının parçalarını ayrı ayrı gerçekleştiren bazı yöntemler kullanacağımızı unutmayın, ancak pratikte tokenizer'ı doğrudan girdileriniz üzerinde çağırmalısınız (Bölüm 2'de gösterildiği gibi).

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


Bu tokenizer bir alt kelime tokenizeridir: kelime dağarcığı tarafından temsil edilebilecek tokenlar elde edene kadar kelimeleri böler. Burada transformer kelimesi iki tokena ayrılmıştır: transform ve ##er.

### From tokens to input IDs

Giriş kimliklerine dönüştürme convert_tokens_to_ids() tokenizer yöntemi tarafından gerçekleştirilir:

In [8]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


Bu çıktılar, uygun framework tensörüne dönüştürüldükten sonra, bu bölümde daha önce görüldüğü gibi bir modelin girdileri olarak kullanılabilir.

## Decoding

Kod çözme işlemi tam tersi şekilde gerçekleşir: kelime indislerinden bir dize elde etmek isteriz. Bu decode() metodu ile aşağıdaki gibi yapılabilir:

In [9]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

2024-08-02 06:41:59.051298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 06:41:59.051435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 06:41:59.199975: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using a transformer network is simple


**decode** yönteminin yalnızca dizinleri tekrar belirteçlere dönüştürmekle kalmadığını, aynı zamanda okunabilir bir cümle üretmek için aynı kelimelerin parçası olan belirteçleri bir araya getirdiğini unutmayın. Bu davranış, yeni metin tahmin eden modeller kullandığımızda (bir istemden oluşturulan metin ya da çeviri veya özetleme gibi diziden diziye problemler için) son derece faydalı olacaktır.

Şimdiye kadar bir tokenizer'ın işleyebileceği atomik işlemleri anlamış olmalısınız: tokenizasyon, ID'lere dönüştürme ve ID'leri bir dizeye geri dönüştürme. Ancak, buzdağının sadece görünen kısmını kazıyabildik. Bir sonraki bölümde, yaklaşımımızı sınırlarına götüreceğiz ve bunların üstesinden nasıl geleceğimize bir göz atacağız.